In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [11]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [26]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

[('https://arca.live/b/aiartreal/95255511?p=1',
  '오늘의 테마 : 새해 / Happy new year 2024',
  '머리는차갑게가슴은뜨겁게'),
 ('https://arca.live/b/aiartreal/95254770?p=1', '징크스 (LOL)', '빠다파리'),
 ('https://arca.live/b/aiartreal/95254610?p=1', '이집트 공주', '빠다파리'),
 ('https://arca.live/b/aiartreal/95254402?p=1', '귀여운 서큐버스', '빠다파리'),
 ('https://arca.live/b/aiartreal/95253937?p=1', '우주 악당들', '산체스'),
 ('https://arca.live/b/aiartreal/95253831?p=1',
  '참고 : Ultimate SD Upscale Extension 사용례',
  'RollnGo'),
 ('https://arca.live/b/aiartreal/95253557?p=1',
  '(스압) heart maebari, coat',
  'ㅇㅇ'),
 ('https://arca.live/b/aiartreal/95253516?p=1', '검스 신고 쪼그려앉기', '설로인'),
 ('https://arca.live/b/aiartreal/95252140?p=1', '야짤 올릴곳 없을까여?', '사랑은개나소나'),
 ('https://arca.live/b/aiartreal/95251338?p=1', '유행지난 메카컨셉', '스노우폭스'),
 ('https://arca.live/b/aiartreal/95251332?p=1', '크진 않지만 쌔보이는분들', 'igo'),
 ('https://arca.live/b/aiartreal/95250752?p=1', '속옷 출근', '바코드'),
 ('https://arca.live/b/aiartreal/95250026?p=1', '기괴함 주의 - 웃는 고양이', 'Phil'

In [12]:
post_list = []
for i in range(3):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95291438?p=3', '서술형 랜덤와카 4', '빠다파리'),
 ('https://arca.live/b/aiartreal/95291602?p=3', '운동녀 사복 느낌', 'showmaker11'),
 ('https://arca.live/b/aiartreal/95291705?p=3', '승마', '빠다파리'),
 ('https://arca.live/b/aiartreal/95291783?p=3', 'i2i가 안 돼요', 'ㅇㅇㅇㅇ'),
 ('https://arca.live/b/aiartreal/95291954?p=3', 'Pooh 5-3 수영복 찍먹', 'carmen'),
 ('https://arca.live/b/aiartreal/95292463?p=3', '안이쁜 사진', '비스비스'),
 ('https://arca.live/b/aiartreal/95292807?p=3', '하드한? 해피 뉴 이어', '저기있는곰'),
 ('https://arca.live/b/aiartreal/95293442?p=3', '2.5D 입니다.', 'Amateras'),
 ('https://arca.live/b/aiartreal/95293609?p=3', '이거 어디서 설정 하는거였죠?', 'SDPRO'),
 ('https://arca.live/b/aiartreal/95293728?p=3',
  'DengDeng_EF0.fp16 수영복 찍먹',
  'carmen'),
 ('https://arca.live/b/aiartreal/95294337?p=3', 'take pose', 'aborim'),
 ('https://arca.live/b/aiartreal/95294397?p=3', 'SDXL 몇장', '리스펙트'),
 ('https://arca.live/b/aiartreal/95294882?p=3',
  '2d~2.5d 그림 (Model hash: b0c5380ce3, Model: JA-PKA V3.3-C )',
  '냥이

In [13]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 서술형 랜덤와카 4
중복 : 운동녀 사복 느낌
중복 : 승마
중복 : i2i가 안 돼요
중복 : Pooh 5-3 수영복 찍먹
중복 : 안이쁜 사진
중복 : 하드한? 해피 뉴 이어
중복 : 2.5D 입니다.
중복 : 이거 어디서 설정 하는거였죠?
중복 : DengDeng_EF0.fp16 수영복 찍먹
중복 : take pose
중복 : SDXL 몇장
중복 : 2d~2.5d 그림 (Model hash: b0c5380ce3, Model: JA-PKA V3.3-C )
중복 : 해피뉴이어 2탄(매운맛)
중복 : just bikini
중복 : 메이드 (푸우님모델 5-3)
중복 : 여자는 다다익선
중복 : 배경) 새해) Happppy new Year!
중복 : 세자매
중복 : 기차 , 눈 , 스키복장,
중복 : 흔한 외모의 여동생
중복 : 푸우님의 5-3 찍먹
중복 : 즐거운 연휴 되세요.
중복 : DT값을 찾아서~ hanimix14,15A
중복 : 두 남자
중복 : 순박한 시골 처녀
중복 : 쉿! 그녀가 자고있어요
중복 : 눈오늘날 야외에서
중복 : 아르테미스 리얼 2.1 갖고 계신분
중복 : 밸리댄서(?)...(스압)
중복 : 오랜만에 왓는데 슴슴하네여?
중복 : 여러복장 해피 뉴 이어
중복 : 듀얼 그래픽카드 세팅 해보신 분 계실까요?
중복 : 하나의 프롬 여러 사진
중복 : 수영복(feat hanimix v14 a+ (?))
중복 : 리터치 할려고 1년 그림 공부한 후기
중복 : 계정을 삭제하면 안 되는 이유
중복 : 개성강한 모델 ,juggernaut_reborn, 338b85bc4f
중복 : 오늘 마지막 업로드
중복 : 부산물
중복 : 아니 이건 필름로라인데 왜 이뻐지는거지?
중복 : 건전한 CROPTOP 여러장
중복 : 저스트 비키니
중복 : 1장)간만에 진짜 너무 맘에드는 사진이 나왔어요
중복 : Happy New Year
중복 : 기본 국밥 로라 프롬 추가
중복 : 얼빡샷 찔끔.
중복 : 저도 자기전에 한장...
중복 : 그럼 카카로트2.5D는 지는

Processing: 133 중 114 95351633__오... 멋지다!!!  _  이노프리
failed : https://arca.live/b/aiartreal/95351633?p=1
Processing: 133 중 115 95353275__잠자고 있는 여동생을...(스압)  _  Taizo
Processing: 133 중 116 95353321__클립챔프 연습영상  _  냥이집사
failed : https://arca.live/b/aiartreal/95353321?p=1
Processing: 133 중 117 95354997__super-hentai-fp16-no-ema 부먹  _  빠다파리
failed : https://arca.live/b/aiartreal/95354997?p=1
Processing: 133 중 118 95355353__메카닉 수인 자꾸 멈출수가 없어요  _  스노우폭스
Processing: 133 중 119 95355830__아무도 안궁금해할 2D 애니계에서 살아남는법  _  kkoor
failed : https://arca.live/b/aiartreal/95355830?p=1
Processing: 133 중 120 95355966__super-hentai-fp16-no-ema 제작자님의 말씀대로 찍먹  _  엑시옴
Processing: 133 중 121 95356632__새해에는 꼭 열심히 운동을... (hanimix_v19(26D))  _  귀월랑
Processing: 133 중 122 95357579__NewHorizon 찍먹..  _  레드프레임21
failed : https://arca.live/b/aiartreal/95357579?p=1
Processing: 133 중 123 95357708__바이올리니스트  _  MS07B3
Processing: 133 중 124 95358015__ㅎ  _  Invalid_Syntax
Processing: 133 중 125 95358200__피자 먹고 싶다  _  북이
Processing

In [14]:
list = process_images(driver, list, download_path)

Processing: 14 중 1 95335481__타일 리샘플이 만능은 아니네  _  aswqdfgt
failed : https://arca.live/b/aiartreal/95335481?p=2
Processing: 14 중 2 95336388__기사 (NewHorizon)  _  너예쁘다
Processing: 14 중 3 95339378__엌 해피뉴이어 야짤을 뚫고 나온 그림  _  고삐풀린뉴비
failed : https://arca.live/b/aiartreal/95339378?p=2
Processing: 14 중 4 95344806__NewHorizon 부먹  _  빠다파리
Processing: 14 중 5 95348329__음 다들 이미 이렇게 하고잇는진 몰겟지만 꿀팁 풀고감  _  kkoor
failed : https://arca.live/b/aiartreal/95348329?p=1
Processing: 14 중 6 95349955__뉴비 병합질문 드립니다~  _  바지사장용사
failed : https://arca.live/b/aiartreal/95349955?p=1
Processing: 14 중 7 95350834__????  _  이노프리
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x00000001012104cc chromedriver + 4162764
1   chromedriver                        0x0000000101208654 chromedriver + 4130388
2   chromedriver                        0x0000000100e5fbc0 chromedriver + 293824
3   chromedriver                    

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [8]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [9]:
df = combine_csv_files_remove_duplicates(script_dir)

In [10]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
2212,https://arca.live/b/aiartreal/95302106,쉿! 그녀가 자고있어요
2213,https://arca.live/b/aiartreal/95302742,아르테미스 리얼 2.1 갖고 계신분
2214,https://arca.live/b/aiartreal/95306617,듀얼 그래픽카드 세팅 해보신 분 계실까요?
2215,https://arca.live/b/aiartreal/95307865,계정을 삭제하면 안 되는 이유


['94993242_젬마.zip',
 '94967708_리스펙트.zip',
 '95016252_TeamA.zip',
 '95042883_carmen.zip',
 '95043687_리버스Q.zip',
 '94976087_젬마.zip',
 'civitai_Day_20231225_070258.xlsx',
 '94958822_너예쁘다.zip',
 '95061365_호릉불.zip',
 '95060974_초콜릿주세요.zip',
 '94935829_리스펙트.zip',
 '95056954_ㅇㅇ.zip',
 '95000214_Gbox.zip',
 '95013973_야발야발.zip',
 '95069262_Invalid_Syntax.zip',
 '95057424_바코드.zip',
 '95068665_coffee7897.zip',
 '94987201_머리는차갑게가슴은뜨겁게.zip',
 '94973800_빠다파리.zip',
 '95056478_리버스Q.zip',
 '95001410_thec.zip',
 '95046524_스노우폭스.zip',
 '94965605_냥이집사.zip',
 '94977635_showmaker11.zip',
 '.DS_Store',
 '95042712_carmen.zip',
 '94995036_머리는차갑게가슴은뜨겁게.zip',
 '94942780_그림꾼달잡이.zip',
 '94958197_Ai우유.zip',
 '94995125_빠다파리.zip',
 '95057361_파르나.zip',
 '94996914_사랑은개나소나.zip',
 '94973976_빠다파리.zip',
 '94943294_빠다파리.zip',
 '94954341_리스펙트.zip',
 '95056949_머리는차갑게가슴은뜨겁게.zip',
 'url_data.csv',
 '94926326_그림꾼달잡이.zip',
 '95017531_야발야발.zip',
 '95023245_yong91892.zip',
 '95063411_그림꾼달잡이.zip',
 '94992489_하니타카.zip',
 '94975930

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            